In [ ]:
import graphviz
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display, Image
import os
import tempfile
from spin_direct.translation import SPINtoPRISM, TransitionType

In [ ]:
def create_complex_example_model():
    """Create a more complex SPIN model demonstrating all transition types"""
    model = SPINtoPRISM()

    model.add_place("cutting_p0", 1, is_initial=True)
    model.add_transition("cutting_t0", TransitionType.TASK,
                         ["cutting_p0"], ["cutting_done_p1"],
                         impact_vector=[10, 1])
    model.add_place("cutting_done_p1", 0)

    model.add_transition("parallel_t1", TransitionType.PARALLEL_SPLIT,
                         ["cutting_done_p1"], ["bending_p2", "milling_p3"])

    model.add_place("bending_p2", 1)
    model.add_transition("bending_t2", TransitionType.TASK,
                         ["bending_p2"], ["bending_done_p4"],
                         impact_vector=[20, 1])
    model.add_place("bending_done_p4", 0)

    model.add_transition("sequential_t4", TransitionType.SINGLE,
                         ["bending_done_p4"], ["nature1_p6"])

    model.add_place("nature1_p6", 0)
    model.add_place("nature1_end_p16", 0)

    model.add_transition("nature1_t6_7", TransitionType.NATURE,
                         ["nature1_p6"], ["heavy_polish_p8", "light_polish_p9"],
                         probability=0.2)  # 20% chance needs heavy polishing

    model.add_place("heavy_polish_p8", 2)
    model.add_transition("heavy_polish_t10", TransitionType.TASK,
                         ["heavy_polish_p8"], ["heavy_polish_p12"],
                         impact_vector=[5, 4])
    model.add_place("heavy_polish_p12", 0)

    model.add_place("light_polish_p9", 1)
    model.add_transition("light_polish_t11", TransitionType.TASK,
                         ["light_polish_p9"], ["light_polish_p13"],
                         impact_vector=[8, 1])
    model.add_place("light_polish_p13", 0)

    #End nature
    model.add_transition("nature1_decision_0_t14", TransitionType.SINGLE,
                         ["heavy_polish_p12"], ["nature1_end_p16"])
    model.add_transition("nature1_decision_1_t15", TransitionType.SINGLE,
                         ["light_polish_p13"], ["nature1_end_p16"])

    model.add_place("milling_p3", 1)
    model.add_transition("milling_t3", TransitionType.TASK,
                         ["milling_p3"], ["milling_done_p5"],
                         impact_vector=[50, 1])
    model.add_place("milling_done_p5", 0)

    model.add_transition("sequential_t5", TransitionType.SINGLE,
                         ["milling_done_p5"], ["choice1_p7"])

    model.add_place("choice1_p7", 0)
    model.add_transition("choice1_t8_9", TransitionType.CHOICE,
                         ["choice1_p7"], ["fine_depos_p10", "rough_depos_p11"])
    model.add_place("choice1_end_p17", 0)


    model.add_place("fine_depos_p10", 1)
    model.add_transition("fine_depos_t12", TransitionType.TASK,
                         ["fine_depos_p10"], ["fine_depos_p14"],
                         impact_vector=[30, 1])
    model.add_place("fine_depos_p14", 0)

    model.add_place("rough_depos_p11", 1)
    model.add_transition("rough_depos_t13", TransitionType.TASK,
                         ["rough_depos_p11"], ["rough_depos_p15"],
                         impact_vector=[10, 1])
    model.add_place("rough_depos_p15", 0)

    #End choice
    model.add_transition("choice1_decision_0_t16", TransitionType.SINGLE,
                         ["fine_depos_p14"], ["choice1_end_p17"])
    model.add_transition("choice1_decision_1_t17", TransitionType.SINGLE,
                         ["rough_depos_p15"], ["choice1_end_p17"])

    model.add_transition("parallel_merge_t18", TransitionType.PARALLEL_MERGE,
                         ["nature1_end_p16", "choice1_end_p17"], ["choice2_p18"])

    model.add_place("choice2_p18", 0)
    model.add_place("choice2_end_p23", 0)
    model.add_transition("choice2_t19_20", TransitionType.CHOICE,
                         ["choice2_p18"], ["hphs_p19", "lpls_p20"])

    model.add_place("hphs_p19", 1)
    model.add_transition("hphs_t21", TransitionType.TASK,
                         ["hphs_p19"], ["hphs_p21"],
                         impact_vector=[40, 1])
    model.add_place("hphs_p21", 0)
    model.add_transition("choice1_decision_0_t23", TransitionType.SINGLE,
                         ["hphs_p21"], ["choice2_end_p23"])

    model.add_place("lpls_p20", 2)
    model.add_transition("lpls_t22", TransitionType.TASK,
                         ["lpls_p20"], ["lpls_p22"],
                         impact_vector=[20, 3])
    model.add_place("lpls_p22", 0)
    model.add_transition("choice1_decision_1_t24", TransitionType.SINGLE,
                         ["lpls_p22"], ["choice2_end_p23"])


    return model

In [ ]:

def visualize_and_display(model):
    """Create, render and display the SPIN visualization"""
    
    # Create the graphviz object with better formatting
    dot = graphviz.Digraph(comment='SPIN Model', format='png')
    dot.attr(rankdir='LR', size='12,8', dpi='300')
    dot.attr('node', fontsize='10', fontname='Arial')
    dot.attr('edge', fontsize='8', fontname='Arial')
    
    # Add places with better styling
    for name, place in model.places.items():
        if place.is_initial:
            shape = 'doublecircle'
            color = 'lightblue'
            style = 'filled'
        else:
            shape = 'circle'
            color = 'lightgray'
            style = 'filled'
            
        label = f"{name}\\nd={place.duration}"
        dot.node(name, label, shape=shape, color=color, style=style)
        
    # Add transitions and connections with better styling
    for transition in model.transitions:
        # Create transition node with type-specific styling
        if transition.type == TransitionType.NATURE:
            label = f"{transition.name}\\np={transition.probability}"
            color = 'lightcoral'
            style = 'filled'
        elif transition.type == TransitionType.CHOICE:
            label = f"{transition.name}\\n(choice)"
            color = 'lightblue'
            style = 'filled'
        elif transition.type == TransitionType.TASK:
            label = f"{transition.name}\\n{transition.impact_vector}"
            color = 'lightgreen'
            style = 'filled'
        elif transition.type == TransitionType.PARALLEL_SPLIT:
            label = f"{transition.name}\\n(||split)"
            color = 'lightyellow'
            style = 'filled'
        elif transition.type == TransitionType.PARALLEL_MERGE:
            label = f"{transition.name}\\n(||merge)"
            color = 'lightyellow'
            style = 'filled'
        else:
            label = transition.name
            color = 'white'
            style = 'filled'
            
        t_node = f"t_{transition.name}"
        dot.node(t_node, label, shape='box', color=color, style=style)
        
        # Add edges from input places to transition
        for p_in in transition.input_places:
            dot.edge(p_in, t_node, arrowsize='0.7')
            
        # Add edges from transition to output places
        for p_out in transition.output_places:
            dot.edge(t_node, p_out, arrowsize='0.7')
    
    # Render the graph
    try:
        # Create a temporary directory
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file = os.path.join(temp_dir, 'spin_model')
            
            # Render to file
            rendered_file = dot.render(temp_file, cleanup=True)
            
            # Display the image
            print("SPIN Model Visualization:")
            print("="*50)
            
            # Try to display in different ways depending on environment
            try:
                # For Jupyter notebooks
                from IPython.display import Image as IPImage, display
                display(IPImage(rendered_file))
            except ImportError:
                try:
                    # For regular Python with matplotlib
                    img = mpimg.imread(rendered_file)
                    plt.figure(figsize=(15, 10))
                    plt.imshow(img)
                    plt.axis('off')
                    plt.title('SPIN Model Visualization', fontsize=16)
                    plt.tight_layout()
                    plt.show()
                except Exception as e:
                    print(f"Could not display image with matplotlib: {e}")
                    print(f"Image saved to: {rendered_file}")
            
            print(f"\nGraph successfully rendered!")
            print(f"Nodes: {len(model.places)} places + {len(model.transitions)} transitions")
            print(f"Edges: {sum(len(t.input_places) + len(t.output_places) for t in model.transitions)}")
            
    except Exception as e:
        print(f"Error rendering graph: {e}")
        print("Falling back to DOT source code:")
        print(dot.source)
    
    return dot

# Create and visualize the model
print("Creating and visualizing SPIN model...")
model = create_complex_example_model()

# Print model summary
print("\n" + "="*60)
print("MODEL SUMMARY")
print("="*60)
model.print_model_summary()

# Visualize
print("\n" + "="*60)
print("VISUALIZATION")
print("="*60)
viz = visualize_and_display(model)

# Also show the DOT source for reference
print("\n" + "="*60)
print("DOT SOURCE CODE")
print("="*60)
print(viz.source)

In [ ]:
print(model.generate_prism_model())

In [ ]:
with open("example.nm", "w") as f:
    f.write(model.generate_prism_model())